In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import RegionMocked, read_true_geometry, read_true_metadata
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

In [2]:
# read_true_geometry("region_a").pipe(to_geopandas).explore()

In [15]:
# read_true_geometry("region_a").pipe(to_geopandas).explore()
# read_true_geometry("region_b").pipe(to_geopandas).explore()
# read_true_geometry("region_c").pipe(to_geopandas).explore()

In [23]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_b"),
    geometry_to=read_true_geometry("region_c"),
)
intersection_area.pipe(print)
intersection_area.rows(named=True)

shape: (11, 3)
┌──────────┬──────────┬─────────┐
│ region_b ┆ region_c ┆ mapping │
│ ---      ┆ ---      ┆ ---     │
│ str      ┆ str      ┆ f64     │
╞══════════╪══════════╪═════════╡
│ A        ┆ X        ┆ 14.0    │
│ A        ┆ Y        ┆ 16.0    │
│ A        ┆ Z        ┆ 2.0     │
│ B        ┆ X        ┆ 1.0     │
│ B        ┆ Y        ┆ 8.0     │
│ …        ┆ …        ┆ …       │
│ C        ┆ X        ┆ 1.0     │
│ C        ┆ Y        ┆ 8.0     │
│ C        ┆ Z        ┆ 7.0     │
│ C        ┆ null     ┆ 16.0    │
│ B        ┆ null     ┆ 16.0    │
└──────────┴──────────┴─────────┘


[{'region_b': 'A', 'region_c': 'X', 'mapping': 14.0},
 {'region_b': 'A', 'region_c': 'Y', 'mapping': 16.0},
 {'region_b': 'A', 'region_c': 'Z', 'mapping': 2.0},
 {'region_b': 'B', 'region_c': 'X', 'mapping': 1.0},
 {'region_b': 'B', 'region_c': 'Y', 'mapping': 8.0},
 {'region_b': 'B', 'region_c': 'Z', 'mapping': 7.0},
 {'region_b': 'C', 'region_c': 'X', 'mapping': 1.0},
 {'region_b': 'C', 'region_c': 'Y', 'mapping': 8.0},
 {'region_b': 'C', 'region_c': 'Z', 'mapping': 7.0},
 {'region_b': 'C', 'region_c': None, 'mapping': 16.0},
 {'region_b': 'B', 'region_c': None, 'mapping': 16.0}]

In [21]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_a"),
    geometry_to=read_true_geometry("region_c"),
)
intersection_area.pipe(print)
intersection_area.rows(named=True)

shape: (8, 3)
┌──────────┬──────────┬─────────┐
│ region_a ┆ region_c ┆ mapping │
│ ---      ┆ ---      ┆ ---     │
│ str      ┆ str      ┆ f64     │
╞══════════╪══════════╪═════════╡
│ M        ┆ Y        ┆ 8.0     │
│ M        ┆ Z        ┆ 8.0     │
│ N        ┆ Y        ┆ 8.0     │
│ N        ┆ Z        ┆ 8.0     │
│ O        ┆ X        ┆ 8.0     │
│ O        ┆ Y        ┆ 8.0     │
│ P        ┆ X        ┆ 8.0     │
│ P        ┆ Y        ┆ 8.0     │
└──────────┴──────────┴─────────┘


[{'region_a': 'M', 'region_c': 'Y', 'mapping': 8.0},
 {'region_a': 'M', 'region_c': 'Z', 'mapping': 8.0},
 {'region_a': 'N', 'region_c': 'Y', 'mapping': 8.0},
 {'region_a': 'N', 'region_c': 'Z', 'mapping': 8.0},
 {'region_a': 'O', 'region_c': 'X', 'mapping': 8.0},
 {'region_a': 'O', 'region_c': 'Y', 'mapping': 8.0},
 {'region_a': 'P', 'region_c': 'X', 'mapping': 8.0},
 {'region_a': 'P', 'region_c': 'Y', 'mapping': 8.0}]

In [22]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_a"),
    geometry_to=read_true_geometry("region_b"),
)
intersection_area.pipe(print)
intersection_area.rows(named=True)

shape: (10, 3)
┌──────────┬──────────┬─────────┐
│ region_a ┆ region_b ┆ mapping │
│ ---      ┆ ---      ┆ ---     │
│ str      ┆ str      ┆ f64     │
╞══════════╪══════════╪═════════╡
│ M        ┆ A        ┆ 4.0     │
│ M        ┆ B        ┆ 12.0    │
│ N        ┆ A        ┆ 4.0     │
│ N        ┆ C        ┆ 12.0    │
│ O        ┆ A        ┆ 12.0    │
│ O        ┆ B        ┆ 4.0     │
│ P        ┆ A        ┆ 12.0    │
│ P        ┆ C        ┆ 4.0     │
│ null     ┆ B        ┆ 16.0    │
│ null     ┆ C        ┆ 16.0    │
└──────────┴──────────┴─────────┘


[{'region_a': 'M', 'region_b': 'A', 'mapping': 4.0},
 {'region_a': 'M', 'region_b': 'B', 'mapping': 12.0},
 {'region_a': 'N', 'region_b': 'A', 'mapping': 4.0},
 {'region_a': 'N', 'region_b': 'C', 'mapping': 12.0},
 {'region_a': 'O', 'region_b': 'A', 'mapping': 12.0},
 {'region_a': 'O', 'region_b': 'B', 'mapping': 4.0},
 {'region_a': 'P', 'region_b': 'A', 'mapping': 12.0},
 {'region_a': 'P', 'region_b': 'C', 'mapping': 4.0},
 {'region_a': None, 'region_b': 'B', 'mapping': 16.0},
 {'region_a': None, 'region_b': 'C', 'mapping': 16.0}]

In [4]:
intersection_area.to_dict(as_series=False)

{'region_b': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'B', 'C'],
 'region_c': ['X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', None, None],
 'mapping': [1.21875,
  0.5,
  0.28125,
  0.140625,
  0.25,
  0.609375,
  0.140625,
  0.25,
  0.609375,
  1.0,
  1.0]}